In [ ]:
# ================================================================
# 03_kerr_flux_sim.ipynb — Kerr-enhanced quaternionic flux (stable)
# ================================================================
!pip install --quiet scipy

import numpy as np
import matplotlib.pyplot as plt
from  math import sqrt, pi
from scipy.integrate import solve_ivp

# -------- 基本常數 ------------------------------------------------
G, c   = 6.67430e-11, 2.99792458e8
M_sun  = 1.98847e30
M_pl   = 2.176434e-8          # kg

# -------- 黑洞 (Sgr A*) 參數 -------------------------------------
M_bh   = 4.0e6 * M_sun
a_star = 0.5

r_g_m  = 2*G*M_bh/c**2
print(f"r_g ≈ {r_g_m/1e3:.2e} km  ,  a* = {a_star}")

# -------- 無因次化 ------------------------------------------------
a_hat   = a_star
Mratio  = M_bh / M_pl         # ≈ 3.65×10^44
Δ       = lambda x: x**2 - x + a_hat**2
dΔ      = lambda x: 2*x - 1

κ       = 2.0e-2
λ_tilde = lambda x: 2*κ*Mratio / x**3

# -------- ODE -----------------------------------------------------
def rhs(x, y):
    ψ, χ = y
    src  = x**2 * λ_tilde(x) * ψ * (ψ**2 - 1)
    dxd  = (2*x*Δ(x) + x**2*dΔ(x))
    return [χ, (src - dxd*χ) / (x**2 * Δ(x))]

# -------- 積分 (由外往內) -----------------------------------------
x_out  = 100.0
x_in   = 2.0
x_eval = np.geomspace(x_out, x_in, 1000)      # 由外到內

y0 = [0.0, -1.0e-8]        # ψ=0, 微小負斜率 → 破壞平凡解

sol = solve_ivp(rhs, (x_out, x_in), y0,
                t_eval=x_eval,
                method="Radau",
                rtol=1e-8, atol=1e-11,
                max_step=2.0)

if not sol.success:
    raise RuntimeError(sol.message)

# 反轉回遞增 x 方便閱讀
x_arr  = sol.t[::-1]
ψ_arr  = sol.y[0][::-1]
ε_arr  = 2*ψ_arr

# -------- 論文 ansatz 作對照 -------------------------------------
r_s0_hat = (8.5e3 * 3.085677e16) / r_g_m
boost    = 1.0 / (1 - a_hat/x_arr)
ε_ansatz = 2*np.tanh(x_arr / (r_s0_hat * boost))

# -------- 繪圖 ----------------------------------------------------
plt.figure(figsize=(7,5))
plt.loglog(x_arr, ε_arr,   lw=2, label="Numerical ε(r)")
plt.loglog(x_arr, ε_ansatz,'--', lw=2, label="Analytic ansatz")
plt.axvline(10, ls=":", c='k', alpha=.4); plt.text(10.3,1e-2,"10 r_g")
plt.xlabel(r"$\hat r = r/r_g$")
plt.ylabel(r"$\epsilon(r)$")
plt.title("Quaternionic flux in Kerr spacetime (stable integration)")
plt.legend(); plt.grid(ls="--", alpha=0.3); plt.tight_layout()
plt.show()

# -------- δc_g/c 估算 (f = 0.01 Hz) ------------------------------
f_LISA = 1e-2
k_SI   = 2*pi*f_LISA / c
k_hat  = k_SI * r_g_m / c

idx10  = np.argmin(np.abs(x_arr - 10))
ψ10    = ψ_arr[idx10]
deltac = (ψ10**2 * (a_hat/x_arr[idx10])**2) / (2*k_hat**2)

print(f"\n在 10 r_g： ε ≈ {ε_arr[idx10]:.2e}")
print(f"預測 δc_g/c ≈ {deltac:.2e}")
